# 2022.6#1

研究了一些数学基础。包括以下内容：梯度，多元微分，矩阵求导。

函数$f(x)$相对于$x$的梯度是一个包含$n$个偏导数的向量：

$$
\nabla_{\mathbf{x}} f(\mathbf{x})=\left[\frac{\partial f(\mathbf{x})}{\partial x_{1}}, \frac{\partial f(\mathbf{x})}{\partial x_{2}}, \ldots, \frac{\partial f(\mathbf{x})}{\partial x_{n}}\right]^{\top}
$$

**这里有一个定理不太明白。**
假设 $\mathbf{x}$ 为 $n$ 维向量，有:
1. 对于所有 $\mathbf{A} \in \mathbb{R}^{m \times n}$, 都有 $\nabla_{\mathbf{x}} \mathbf{A} \mathbf{x}=\mathbf{A}^{\top}$

    展开后的结果，如果按照上文定义，应该是一个$(mn)\times 1$的矩阵，看见维基上提到实标量函数和m维行向量函数的梯度分别定义，猜测左边计算得到的m维列向量函数应该也是被定义为$n \times m$的矩阵，否则每个函数都会在纵向展开。下面是另外一些定理，都可以通过简单的展开验证。

2. 对于所有 $\mathbf{A} \in \mathbb{R}^{n \times m}$, 都有 $\nabla_{\mathbf{x}} \mathbf{x}^{\top} \mathbf{A}=\mathbf{A}$

    查询资料时发现这条定理应该有更简洁和直观的形式，即A退化为列向量***a***。此时$a^Tx$和$x^Ta$的梯度都是***a***。注意到，若干个列向量的“拼接”即可推广到矩阵的情况，那通过“拼接”能否证明上一定理？现已知$\nabla a^Tx=a$，但$a^T$与$a$扩展方向不同，不能直接验证，反而增加了上述猜想的可能性。
3. 对于所有 $\mathbf{A} \in \mathbb{R}^{n \times n}$, 都有 $\nabla_{\mathbf{x}} \mathbf{x}^{\top} \mathbf{A} \mathbf{x}=\left(\mathbf{A}+\mathbf{A}^{\top}\right) \mathbf{x}$

    此处继续使用硬展开求导有些不明智了。查阅了一些资料，简单研究了一下标量对矩阵求导。所谓梯度就是标量对矩阵求导的结果。借助矩阵的迹，可以将矩阵运算和微分联系起来。
    
    矩阵微分的法则就像标量微分一样，加减法、转置、乘法，形式都非常熟悉。除了链式法则有一些明显的区别，并且需要转置。从矩阵微分转为标量微分需要求内积，或者使用迹运算，这又引入了一些新的规则，可交换的矩阵乘法等等。

    矩阵对矩阵的求导略复杂些，留到下次去填。

总的来说，通过研究矩阵求导和计算一些典型梯度， 我们理解了反向传播函数做了什么工作。对于某一层中的某个参数y，如果它是由上一层中的若干个参数决定的，那么就计算y对这些参数的导数。同时y中已经存储了自终点传播过来的d(target)/dy，根据链式法则就可以得到d(target)/dx。

另外关于backward的实现，查阅到是主要是在C/Cpp层做的工作，python层面几乎都是对参数的处理。这和我的预期相近，过去听说numpy因为能直接操作内存所以速度很快，很好奇为什么python能操作内存，原来也是通过底层的C调用实现。

最后，通常对于硬件区别的解释是，CPU核心数量少，频率高，GPU核心数量多，频率低，适合大规模并行任务。这就容易让人联想到卷积操作，确实是互相独立可并行的。

In [1]:
# 代码方面，在上上周的基础上我们用pytorch的内置组件来代替我们自
# 己写的代码，专注在逻辑而非实现上。
import numpy as np
import torch
from torch.utils import data
from torch import nn

# 沿用了上次的真实模型：
model_w = torch.tensor([2.022, 6.5])
model_b = torch.tensor(4.2)

# 生成数据集的部分保持一致。
def sample_generator(w, b, sample_num):
    # 标准模型：y = wX + b + epsilon
    X = torch.normal(0, 1, (sample_num, len(w)))
    '''
    这里又发现一个奇怪的事，两个形状一样的向量怎么作矩阵乘法呢？如果形状一致，
    X.T @ Y、X @ Y.T、还是 X @ Y，都是一样的结果。
    '''
    y = w @ X.T + b
    # 噪声
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

# features: 500x2, labels: 500x1
features, labels = sample_generator(model_w, model_b, 500)

In [2]:
# 数据集迭代器
batch_size = 50
data_loader = data.DataLoader(data.TensorDataset(*(features, labels)), batch_size, shuffle=True)

In [9]:
# 定义模型，一个全连接层
# 输入2两个参数，输出一个
net = nn.Sequential(nn.Linear(2, 1))

# 初始化参数，下标方位对应的层
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.normal_(0, 0.01)

# 定义损失函数
loss = nn.MSELoss()

# 定义优化算法
trainer = torch.optim.SGD(net.parameters(), lr=0.01)

In [13]:
# 进行训练

# 10个周期
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_loader:
        # 取出一个batch，计算损失
        l = loss(net(X) ,y)
        # 清除梯度并重新计算
        trainer.zero_grad()
        l.backward()
        # 根据梯度和学习率进行优化
        trainer.step()
    l = loss(net(features), labels)
    print(f'第{epoch + 1}个周期, 损失{l:f}')

第1个周期, 损失0.027932
第2个周期, 损失0.018694
第3个周期, 损失0.012530
第4个周期, 损失0.008413
第5个周期, 损失0.005662
第6个周期, 损失0.003824
第7个周期, 损失0.002594
第8个周期, 损失0.001770
第9个周期, 损失0.001218
第10个周期, 损失0.000849


有了上次的经验，整个过程还算比较清晰。疑点：
 - Sequential作为“层”结构的对象具体行为并不明确。在接下来的多层神经网络中希望能得到解答。
 - *这大概不算一个疑点。pytorch实现中的各种引用非常反直觉，在训练过程中，`l`是计算得到的损失，但调用`l`的反向传播函数可以直接改变变量`X，y`的梯度；网络的参数在`net`变量中，但调用`trainer`的`step`方法可以直接更改。虽然便利，但很不习惯。